
# Preparação e Análise de Dados para Redes Neurais


## Objetivo

Explorar separabilidade de classes em 2D, projetar dados 5D para 2D com PCA e preparar o dataset **Spaceship Titanic** para redes neurais com ativação `tanh`.


## Exercício 1 — Dados 2D (4 classes)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

N = 100
params = {
    0: {"mean": [2, 3],  "std": [0.8, 2.5]},
    1: {"mean": [5, 6],  "std": [1.2, 1.9]},
    2: {"mean": [8, 1],  "std": [0.9, 0.9]},
    3: {"mean": [15, 4], "std": [0.5, 2.0]},
}

Xs, ys = [], []
for c, p in params.items():
    mean = np.array(p["mean"])
    std = np.array(p["std"])
    Xc = np.random.randn(N, 2) * std + mean
    Xs.append(Xc)
    ys.append(np.full(N, c))

X = np.vstack(Xs)
y = np.hstack(ys)

plt.figure(figsize=(7,5))
for c in params.keys():
    plt.scatter(X[y==c,0], X[y==c,1], s=12, label=f"Classe {c}", alpha=0.8)
plt.xlabel("x1"); plt.ylabel("x2")
plt.title("Exercício 1 — Distribuição 2D (4 classes)")
plt.legend()
plt.show()



**Discussão:**  
As classes formam nuvens gaussianas distintas em *x1*, mas com diferentes dispersões em *x2*.  
Uma única fronteira linear não separa as quatro classes; é necessário múltiplas fronteiras ou um modelo não linear.


In [ ]:

# Superfícies de decisão com MLP
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(16,), activation="tanh", max_iter=2000, random_state=42)
clf.fit(X, y)

xx, yy = np.meshgrid(
    np.linspace(X[:,0].min()-1, X[:,0].max()+1, 300),
    np.linspace(X[:,1].min()-1, X[:,1].max()+1, 300),
)
ZZ = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(7,5))
plt.contourf(xx, yy, ZZ, alpha=0.25, levels=[-0.5,0.5,1.5,2.5,3.5])
for c in params.keys():
    plt.scatter(X[y==c,0], X[y==c,1], s=10, label=f"Classe {c}")
plt.xlabel("x1"); plt.ylabel("x2")
plt.title("Exercício 1 — Superfícies de decisão (MLP tanh)")
plt.legend()
plt.show()


## Exercício 2 — Dados 5D (A/B) + PCA(2D)

In [ ]:

from sklearn.decomposition import PCA

np.random.seed(7)

muA = np.array([0, 0, 0, 0, 0])
SigmaA = np.array([
    [1.0, 0.8, 0.1, 0.0, 0.0],
    [0.8, 1.0, 0.3, 0.0, 0.0],
    [0.1, 0.3, 1.0, 0.5, 0.0],
    [0.0, 0.0, 0.5, 1.0, 0.2],
    [0.0, 0.0, 0.0, 0.2, 1.0]
])

muB = np.array([1.5, 1.5, 1.5, 1.5, 1.5])
SigmaB = np.array([
    [1.5, -0.7, 0.2, 0.0, 0.0],
    [-0.7, 1.5, 0.4, 0.0, 0.0],
    [0.2, 0.4, 1.5, 0.6, 0.0],
    [0.0, 0.0, 0.6, 1.5, 0.3],
    [0.0, 0.0, 0.0, 0.3, 1.5]
])

XA = np.random.multivariate_normal(muA, SigmaA, size=500)
XB = np.random.multivariate_normal(muB, SigmaB, size=500)
X5 = np.vstack([XA, XB])
y5 = np.hstack([np.zeros(500), np.ones(500)])

pca = PCA(n_components=2, random_state=42)
X2 = pca.fit_transform(X5)

plt.figure(figsize=(7,5))
plt.scatter(X2[y5==0,0], X2[y5==0,1], s=10, label="Classe A")
plt.scatter(X2[y5==1,0], X2[y5==1,1], s=10, label="Classe B")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.title("Exercício 2 — PCA (5D → 2D)")
plt.legend()
plt.show()



**Discussão:**  
As classes A e B apresentam deslocamento médio, mas as covariâncias geram sobreposição.  
Um perceptron simples não resolve bem; redes com não-linearidades modelam melhor essas fronteiras.


## Exercício 3 — Spaceship Titanic: pré-processamento

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import os

csv_path = "data/spaceship_titanic.csv"

if not os.path.exists(csv_path):
    print("Baixe o dataset e coloque em", csv_path)
else:
    df = pd.read_csv(csv_path)
    target_col = "Transported"
    num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
    cat_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Cabin"]

    df[target_col] = df[target_col].astype(int)

    X = df[num_cols + cat_cols]
    y = df[target_col]

    num_transform = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    cat_transform = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    pre = ColumnTransformer([
        ("num", num_transform, num_cols),
        ("cat", cat_transform, cat_cols)
    ])

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    Xt = pre.fit_transform(X_train)
    Xv = pre.transform(X_val)

    print("Shape treino:", Xt.shape, " Shape validação:", Xv.shape)

    # Histogramas antes/depois para Age
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,3.5))
    ax1.hist(df["Age"].dropna(), bins=40)
    ax1.set_title("Age — original")
    scaled = StandardScaler().fit_transform(df[["Age"]].fillna(df["Age"].median()))
    ax2.hist(scaled, bins=40)
    ax2.set_title("Age — escalado")
    plt.tight_layout()
    plt.show()



**Discussão:**  
- Variáveis numéricas foram padronizadas (média 0, desvio 1) para melhor desempenho com `tanh`.  
- Variáveis categóricas receberam imputação de valores ausentes e codificação *one-hot*.  
- A estratégia de imputação foi **mediana** para numéricos e **mais frequente** para categóricos.


## Conclusões

Os experimentos mostraram que, em dados sintéticos 2D, fronteiras lineares são insuficientes, sendo necessário o uso de funções de ativação não lineares. Nos dados 5D projetados em 2D, a presença de correlações reforça essa limitação dos modelos lineares. Já no caso do Spaceship Titanic, o pré-processamento adequado — com imputação, codificação e padronização — é essencial para que redes com `tanh` operem de forma estável e eficiente.